# Consume Messages in Plain JSON

In [ ]:
import env
import json
# Import the required library
from confluent_kafka import DeserializingConsumer
from confluent_kafka.serialization import SerializationContext, MessageField, StringSerializer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer

# Configuring the Kafka consumer
conf = {
    "bootstrap.servers": env.KAFKA_SERVICE_URI,
    "security.protocol": "SSL",
    "ssl.ca.location": env.CERTIFICATES_FOLDER + "/ca.pem",
    "ssl.certificate.location": env.CERTIFICATES_FOLDER + "/service.cert",
    "ssl.key.location": env.CERTIFICATES_FOLDER + "/service.key",
    'group.id': 'demo-consumer-group-1',
    'auto.offset.reset': 'earliest'
}

consumer = DeserializingConsumer(
    conf
)

# Subscribe to the Kafka topic
consumer.subscribe([env.TOPIC_PREFIX])

# Consume messages from the Kafka topic
while True:
    try:
        msg = consumer.poll(1.0)

        if msg is None:
            continue

        if msg.error():
            print("Consumer error: {}".format(msg.error()))
            continue
        
        id = json.loads(msg.key().decode('utf-8'))["id"]
        name = json.loads(msg.value().decode('utf-8'))["name"]
        age = json.loads(msg.value().decode('utf-8'))["age"]

        if id is not None and name is not None and age is not None:
            print(
                "id:   --> {}\n"
                "name: --> {}\n"
                "age:  --> {}\n"
                "------------------------------".format(id, name, age)
            )

    except KeyboardInterrupt:
            break

    # Commit the offset for the message
    consumer.commit()